In [1]:
from transformers import pipeline
import spacy
import en_core_web_sm
import nltk
from goose3 import Goose
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge

In [2]:
summarizor = pipeline('summarization', model = 'google/pegasus-xsum')

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

In [5]:
tokenizer.model_max_length

512

In [6]:
file = "C:\\Lachin\\NLP Project\\Full podcast.txt"
with open(file,'r') as f:
    original_text = f.readlines()
original_text = ''.join(original_text)

In [7]:
file1 = "C:\\Lachin\\NLP Project\\reference_summary.txt"
with open(file1,'r') as f1:
    reference_summary = f1.readlines()
reference_summary = ''.join(reference_summary)

In [10]:
def summarizing_text(text, number_of_tokens = tokenizer.model_max_length, max_len=0, min_len=0, human_summary=reference_summary):
    sentences = []
    total_sentences = []
    length = 0
    original_sentences = nltk.tokenize.sent_tokenize(text)
    for sent in original_sentences:
        word_tokens = tokenizer(sent)
        length += len(word_tokens['input_ids'])
        if (length < number_of_tokens):
            sentences.append(sent)
        else:
            total_sentences.append(sentences)
            length = len(word_tokens['input_ids'])
            sentences = []
            sentences.append(sent)
    
    
    #Summarizing the text now
    system_summary = []
    for i in range(len(total_sentences)):
        if (i==0):
            print(len(total_sentences))
        print(i)
        text1 = ' '.join(total_sentences[i])
        if ((max_len==0) and (min_len==0)):
            z = summarizor(text1, do_sample=False)
            system_summary.append(z[0]['summary_text'])
        elif((max_len >0) and (min_len > 0)):
            z = summarizor(text1, max_length=max_len, min_length =min_len, do_sample=False)
            system_summary.append(z[0]['summary_text'])
        else:
            print('Please check "max_len" and "min_len" parameters')
    final_system_summary = " ".join(system_summary)
    final_summary_tokens = tokenizer(final_system_summary)
    print('Number of tokens in system summary {}'.format(len(final_summary_tokens['input_ids'])))
    human_summary_tokens = tokenizer(human_summary)
    print('Number of tokens in human summary {}'.format(len(human_summary_tokens['input_ids'])))
    
    #Accuracy checking
    r = Rouge()
    results = r.get_scores(final_system_summary,human_summary)
    print(results)
    
    
    print(final_system_summary)
        
        

In [12]:
%%time
summarizing_text(original_text)

14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Number of tokens in system summary 392
Number of tokens in human summary 503
[{'rouge-1': {'r': 0.24434389140271492, 'p': 0.421875, 'f': 0.3094555827475965}, 'rouge-2': {'r': 0.08831168831168831, 'p': 0.1559633027522936, 'f': 0.11276948128731706}, 'rouge-l': {'r': 0.23076923076923078, 'p': 0.3984375, 'f': 0.29226360567023263}}]
In our series of letters from entrepreneurs, Harj Taggar looks at why you should have a co-founder when you're starting a company. The first reason I'd say to have a co-founder when you're starting a company is that you're more likely to have a co-founder who's invested in the success of the company. In our interview with Paul Graham, the founder of Y Combinator, he explains why he doesn't fund companies founded by two people: Here's what Dropbox co-founder Drew Houston had to say about bringing on a co-founder: The first thing you want to look for in a co-founder is whether or not they've been through a tough time. If you want

In [13]:
%%time
summarizing_text(original_text,number_of_tokens=512, max_len=50,min_len=25)

14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Number of tokens in system summary 435
Number of tokens in human summary 503
[{'rouge-1': {'r': 0.248868778280543, 'p': 0.4198473282442748, 'f': 0.31249999532686606}, 'rouge-2': {'r': 0.07792207792207792, 'p': 0.13333333333333333, 'f': 0.0983606510816987}, 'rouge-l': {'r': 0.23076923076923078, 'p': 0.3893129770992366, 'f': 0.28977272259959336}}]
In our series of letters from entrepreneurs, Harj Taggar looks at why you should have a co-founder when you're starting a company. The first reason I'd say to have a co-founder when you're starting a company is that you're more likely to have a co-founder who's invested in the success of the company. In our interview with Paul Graham, the founder of Y Combinator, he explains why he doesn't fund companies founded by two people: Here's what Dropbox co-founder Drew Houston had to say about bringing on a co-founder to start a company: The first thing you want to look for in a co-founder is whether or not they've b